Test for using actionable-recorurse, provided on https://github.com/ustunb/actionable-recourse

In order to compare recourse for several similar classifiers, we use cross validation to fit several logistic regression models (Is this the right way?). In the next step, we want to check whether the flipsets generated for one of them apply also for the other classifiers.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
import recourse as rs
from recourse.builder import ActionSet #FIX
from recourse.flipset import Flipset #FIX
from recourse.auditor import RecourseAuditor #FIX

import data

In [2]:
url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
df = pd.read_csv(url, skipinitialspace=True)
y, X = df.iloc[:, 0], df.iloc[:, 1:]

NEW: Use Cross validation to train several different classifiers

In [3]:
clf = LogisticRegression(max_iter=10000)
cv = cross_validate(clf, X, y, cv=20, return_estimator=True)

NEW: Select those classifiers that achieve performance within certain tolerance

In [4]:
#X_test = X[:3]
#for i, est in enumerate(scores['estimator']):
#    print(scores['test_score'][i], est.predict(X_test))
cv_scores = cv['test_score']
tolerance = 2*np.std(cv_scores)
good_classifiers = np.array(cv['estimator'])[cv_scores >= np.max(cv_scores) - tolerance]

print(np.max(cv_scores))
print(cv_scores.std())
print(len(good_classifiers))

0.826
0.009025580929287106
9


get predictions

In [5]:
yhat = [clf.predict(X) for clf in good_classifiers]

customize the set of actions

In [6]:
A = ActionSet(X)  ## matrix of features. ActionSet will learn default bounds and step-size.

specify immutable variables

In [7]:
A['Married'].mutable = False

can only specify properties for multiple variables using a list

In [8]:
A[['Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60']].mutable = False

education level

In [9]:
A['EducationLevel'].step_direction = 1  ## force conditional immutability.
A['EducationLevel'].step_size = 1  ## set step-size to a custom value.
A['EducationLevel'].step_type = "absolute"  ## force conditional immutability.
A['EducationLevel'].bounds = (0, 3)

In [10]:
A['TotalMonthsOverdue'].step_size = 1  ## set step-size to a custom value.
A['TotalMonthsOverdue'].step_type = "absolute"  ## discretize on absolute values of feature rather than percentile values
A['TotalMonthsOverdue'].bounds = (0, 100)  ## set bounds to a custom value.

get model coefficients and align
**CONCERN:** Does this actually lead to different actionsets or is one aligned over and over?
What does align actually do with the action set?

In [22]:
action_sets = [A for clf in good_classifiers]  ## tells `ActionSet` which directions each feature should move in to produce positive change.
for i in range(len(action_sets)):
    action_sets[i].align(good_classifiers[i])

Get one individual
**TODO:** We here do this for one classifier. Goal: Do it for all?

In [24]:
j_clf = 0
i = np.flatnonzero(yhat[j_clf] <= 0)[0]

build a flipset for one individual

In [25]:
%%capture 
#discard the output of this cell (fs.populate prints way to much)
fs = Flipset(x = [X.iloc[i]], action_set = action_sets[j_clf], clf = good_classifiers[j_clf])
fs.populate(enumeration_type = 'distinct_subsets', total_items = 10)

**TODO:** How can we test whether the Flipset actually changes the outcome?

In [26]:
print(fs.to_latex())

\begin{tabular}{rlccc}
\toprule
 &          \textsc{Feature Subset} &  \textsc{Current Values} &               &  \textsc{Required Values} \\
0    &               \textit{TotalMonthsOverdue} &                      7.0 &  $\longrightarrow$ &                       3.0 \\
1    &  \textit{MaxPaymentAmountOverLast6Months} &                    100.0 &  $\longrightarrow$ &                     110.0 \\
1    &               \textit{TotalMonthsOverdue} &                      7.0 &  $\longrightarrow$ &                       3.0 \\
2    &             \textit{MostRecentBillAmount} &                   2010.0 &  $\longrightarrow$ &                    1926.0 \\
2    &               \textit{TotalMonthsOverdue} &                      7.0 &  $\longrightarrow$ &                       3.0 \\
3    &          \textit{MostRecentPaymentAmount} &                    100.0 &  $\longrightarrow$ &                     105.0 \\
3    &               \textit{TotalMonthsOverdue} &                      7.0 &  $\longright

**TODO:** Adapt from here to end: (How) do we want t use the auditor?

Run Recourse Audit on Training Data

In [27]:
auditor = RecourseAuditor(action_sets[j_clf], coefficients = good_classifiers[j_clf].coef_[0], intercept = good_classifiers[j_clf].intercept_[0])
audit_df = auditor.audit(X)  ## matrix of features over which we will perform the audit.

print mean feasibility and cost of recourse

In [28]:
print(audit_df['feasible'].mean())
print(audit_df['cost'].mean())

1.0
0.044684389267020806


**TODO**: Generate flipsets for each good classifier and test whether it changes the outcame when the ather good classifiers are used.